<a href="https://www.kaggle.com/code/vtrackstar/2025-fantasy-basketball-team-power-rankings?scriptVersionId=269782300" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [328]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install espn-api

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from espn_api.basketball import League

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preseason202526stats/preseason.csv


In [329]:
# League Info
league_id = 2072742117
season_id = 2025
swid = '{A5D0418F-EB37-4A5B-9041-8FEB372A5BCC}'
espn_s2 = 'AECpVxhDRvfrKeWJdI9zJkUUbpoBPS5ybsG8SLFsL%2BIB%2ByM0zKt0xT2qU%2Bhoa%2B05NvvO%2BSkz3SK7VkToD4Yw5fg89Vrj8qPB%2FGHC3W81lJ9bWPTUw3Ss9YSFuYUIkorKnXJBs3YefGPPo4n9SabSdlY0YbEx2oQ%2Bdls8e3JcKMVHXf4j9ER3v1qY406MEWmlIc0Ubw2u7cEjs1yUGWlOPfMcZml%2BTUkTVF6HJhEjzWQ6bhOGUHEif8KfpJ4TY1ydkf8l%2F5JycpD9m1cQ8bP9f4g5b5EqmF4U0ZxHrM6FFc%2FPSw%3D%3D'

league = League(league_id=league_id, year=season_id, espn_s2=espn_s2, swid=swid)

In [330]:
# --- PROCESS EACH TEAM ---
team_dfs = {}  # store team DataFrames here

for team in league.teams:
    team_name = team.team_name
    rows = []

    for player in team.roster:
        # player.stats is a nested dict (e.g., {'2025_total': {...}, '2025_last_7': {...}})
        total_stats = player.stats.get('2025_total', {}).get('avg', {})  # avg or total depending on what you want
        total_stats['player'] = player.name
        total_stats['position'] = player.position
        rows.append(total_stats)

    # Create DataFrame for the team
    df = pd.DataFrame(rows)
    df = df[['player', 'position'] + [col for col in df.columns if col not in ['player', 'position']]]
    team_dfs[team_name] = df

    # Print nicely formatted table
    print(f"\n===== {team_name} =====")
    print(df.to_string(index=False))

    # Optionally save each team to a CSV file
    df.to_csv(f"{team_name.replace(' ', '_')}.csv", index=False)


===== Geriatric Knee's and Buckets =====
               player position       PTS      BLK      STL       AST     OREB     DREB       REB  EJ       FF       PF       TF       TO  DQ       FGM       FGA      FTM       FTA      3PM      3PA      FG%      FT%     3PT%     AFG%      FGMI     FTMI     3PMI       APG      BPG       MPG       PPG       RPG      SPG     TOPG      3PG      PPM     A/TO      STR       DD       TD  QD       MIN       GS   GP       TW      FTR
Giannis Antetokounmpo       PF 30.388060 1.164179 0.865672  6.462687 2.194030 9.716418 11.910448 0.0 0.000000 2.313433 0.014925 3.074627 0.0 11.835821 19.686567 6.507463 10.552239 0.208955 0.940299 0.601213 0.616690 0.222222 0.606520  7.850746 4.044776 0.731343  6.462687 1.164179 34.194030 30.388060 11.910448 0.865672 3.074627 0.208955 0.888695 2.101942 0.281553 0.820896 0.164179 0.0 34.194030 1.000000 67.0 0.597015 0.536012
           Trae Young       PG 24.223684 0.157895 1.197368 11.578947 0.473684 2.631579  3.105263 0.0

In [331]:
# --- GATHER ALL TEAMS INTO ONE MASTER LIST ---
all_rows = []

for team in league.teams:
    team_name = team.team_name

    for player in team.roster:
        # Pull season averages (can change to totals or last_7 if you want)
        stats = player.stats.get('2025_total', {}).get('avg', {})
        # Build player row
        row = {'Team': team_name, 'Player': player.name, 'Position': player.position}
        row.update(stats)
        all_rows.append(row)

# --- CREATE MASTER DATAFRAME ---
df_all = pd.DataFrame(all_rows)

# Optional: reorder columns for readability
cols = ['Team', 'Player', 'Position'] + [col for col in df_all.columns if col not in ['Team', 'Player', 'Position']]
df_all = df_all[cols]

# Print preview
df_all

,Team,Player,Position,PTS,BLK,STL,AST,OREB,DREB,REB,...,DD,TD,QD,MIN,GS,GP,TW,FTR,player,position
0,Geriatric Knee's and Buckets,Giannis Antetokounmpo,PF,30.388060,1.164179,0.865672,6.462687,2.194030,9.716418,11.910448,...,0.820896,0.164179,0.0,34.194030,1.000000,67.0,0.597015,0.536012,Giannis Antetokounmpo,PF
1,Geriatric Knee's and Buckets,Trae Young,PG,24.223684,0.157895,1.197368,11.578947,0.473684,2.631579,3.105263,...,0.671053,0.000000,0.0,36.026316,1.000000,76.0,0.473684,0.407703,Trae Young,PG
2,Geriatric Knee's and Buckets,James Harden,PG,22.810127,0.696203,1.493671,8.696203,0.696203,5.075949,5.772152,...,0.481013,0.037975,0.0,35.316456,1.000000,79.0,0.632911,0.446332,James Harden,PG
3,Geriatric Knee's and Buckets,Desmond Bane,SG,19.231884,0.405797,1.159420,5.347826,0.898551,5.159420,6.057971,...,0.144928,0.014493,0.0,31.927536,0.985507,69.0,0.550725,0.194499,Desmond Bane,SG
4,Geriatric Knee's and Buckets,Evan Mobley,PF,18.535211,1.591549,0.859155,3.183099,2.309859,6.971831,9.281690,...,0.521127,0.000000,0.0,30.563380,1.000000,71.0,0.816901,0.336645,Evan Mobley,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Dev,OG Anunoby,SF,18.000000,0.878378,1.486486,2.216216,1.310811,3.527027,4.837838,...,0.013514,0.000000,0.0,36.567568,1.000000,74.0,0.608108,0.220058,OG Anunoby,SF
103,Dev,Buddy Hield,SG,11.121951,0.280488,0.841463,1.634146,0.621951,2.597561,3.219512,...,0.000000,0.000000,0.0,22.768293,0.268293,82.0,0.585366,0.081425,Buddy Hield,SG
104,Dev,Jonas Valanciunas,C,10.407407,0.567901,0.456790,2.024691,2.345679,5.395062,7.740741,...,0.259259,0.000000,0.0,18.790123,0.259259,81.0,0.283951,0.263158,Jonas Valanciunas,C
105,Dev,Scoot Henderson,PG,12.681818,0.196970,1.030303,5.075758,0.787879,2.212121,3.000000,...,0.030303,0.000000,0.0,26.666667,0.151515,66.0,0.454545,0.300730,Scoot Henderson,PG


In [332]:
print(df_all.columns)

Index(['Team', 'Player', 'Position', 'PTS', 'BLK', 'STL', 'AST', 'OREB',
       'DREB', 'REB', 'EJ', 'FF', 'PF', 'TF', 'TO', 'DQ', 'FGM', 'FGA', 'FTM',
       'FTA', '3PM', '3PA', 'FG%', 'FT%', '3PT%', 'AFG%', 'FGMI', 'FTMI',
       '3PMI', 'APG', 'BPG', 'MPG', 'PPG', 'RPG', 'SPG', 'TOPG', '3PG', 'PPM',
       'A/TO', 'STR', 'DD', 'TD', 'QD', 'MIN', 'GS', 'GP', 'TW', 'FTR',
       'player', 'position'],
      dtype='object')


In [333]:
fbball = df_all[['Team', 'Player', 'Position', 'PTS', 'BLK', 'STL', 'AST', 'REB', 'FG%', '3PM', 'FT%']]
fbball

,Team,Player,Position,PTS,BLK,STL,AST,REB,FG%,3PM,FT%
0,Geriatric Knee's and Buckets,Giannis Antetokounmpo,PF,30.388060,1.164179,0.865672,6.462687,11.910448,0.601213,0.208955,0.616690
1,Geriatric Knee's and Buckets,Trae Young,PG,24.223684,0.157895,1.197368,11.578947,3.105263,0.411337,2.868421,0.875223
2,Geriatric Knee's and Buckets,James Harden,PG,22.810127,0.696203,1.493671,8.696203,5.772152,0.410039,2.974684,0.873702
3,Geriatric Knee's and Buckets,Desmond Bane,SG,19.231884,0.405797,1.159420,5.347826,6.057971,0.484283,2.376812,0.893939
4,Geriatric Knee's and Buckets,Evan Mobley,PF,18.535211,1.591549,0.859155,3.183099,9.281690,0.557395,1.197183,0.724590
...,...,...,...,...,...,...,...,...,...,...,...
102,Dev,OG Anunoby,SF,18.000000,0.878378,1.486486,2.216216,4.837838,0.476144,2.310811,0.809735
103,Dev,Buddy Hield,SG,11.121951,0.280488,0.841463,1.634146,3.219512,0.417303,2.475610,0.828125
104,Dev,Jonas Valanciunas,C,10.407407,0.567901,0.456790,2.024691,7.740741,0.550239,0.098765,0.878788
105,Dev,Scoot Henderson,PG,12.681818,0.196970,1.030303,5.075758,3.000000,0.418978,1.590909,0.766990


In [334]:
team_agg = fbball.groupby('Team').mean(numeric_only=True)
team_agg

,PTS,BLK,STL,AST,REB,FG%,3PM,FT%
Team,,,,,,,,
Belt 2 Ass,17.577160,0.767839,1.137369,4.051372,5.662391,0.483913,1.727416,0.792176
Bryan's Brilliant Team,18.814869,0.768053,0.957711,3.400188,6.903414,0.480487,1.971758,0.783600
Da Man Dem,17.922773,0.675186,1.070963,4.394317,6.092572,0.487405,1.933602,0.819880
Dev,17.679109,0.626832,1.024861,3.994457,5.711603,0.464387,1.947325,0.810582
Geriatric Knee's and Buckets,16.464655,0.756204,0.923919,4.689795,6.470292,0.496149,1.504770,0.768385
Keenerson’s Island,18.293471,0.602322,1.091868,4.437019,6.111413,0.480857,2.058953,0.780755
Real Njggas do Real Things,19.563847,0.564840,0.866462,4.021617,6.476871,0.493511,2.186344,0.810669
TytheGreat,18.621570,0.524387,1.158660,4.422630,5.750902,0.478045,1.802105,0.781999


In [335]:
categories = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3PM']

# Create rank columns for each category
ranked = team_agg.copy()
for cat in categories:
    # ascending=False because higher value is better
    ranked[cat + '_rank'] = ranked[cat].rank(ascending=False, method='min')

# Calculate total rank score (sum of ranks)
rank_cols = [c + '_rank' for c in categories if c + '_rank' in ranked.columns]
ranked['TotalRankScore'] = ranked[rank_cols].sum(axis=1)

# Optionally invert so higher = better (makes it easier to read)
ranked['PowerScore'] = len(ranked) * len(categories) - ranked['TotalRankScore'] + 1

# Sort for final power rankings
ranked = ranked.sort_values('PowerScore', ascending=False)

print("=== TEAM POWER RANKINGS BASED ON 2024-25 ===")
print(ranked[['PowerScore', 'TotalRankScore']])

=== TEAM POWER RANKINGS BASED ON 2024-25 ===
                              PowerScore  TotalRankScore
Team                                                    
Real Njggas do Real Things          36.0            29.0
Da Man Dem                          34.0            31.0
Bryan's Brilliant Team              33.0            32.0
Keenerson’s Island                  32.0            33.0
Belt 2 Ass                          26.0            39.0
Geriatric Knee's and Buckets        26.0            39.0
TytheGreat                          25.0            40.0
Dev                                 20.0            45.0


In [336]:
preseason = ranked[['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3PM', 'TotalRankScore']]
preseason

,PTS,REB,AST,STL,BLK,FG%,FT%,3PM,TotalRankScore
Team,,,,,,,,,
Real Njggas do Real Things,19.563847,6.476871,4.021617,0.866462,0.564840,0.493511,0.810669,2.186344,29.0
Da Man Dem,17.922773,6.092572,4.394317,1.070963,0.675186,0.487405,0.819880,1.933602,31.0
Bryan's Brilliant Team,18.814869,6.903414,3.400188,0.957711,0.768053,0.480487,0.783600,1.971758,32.0
Keenerson’s Island,18.293471,6.111413,4.437019,1.091868,0.602322,0.480857,0.780755,2.058953,33.0
Belt 2 Ass,17.577160,5.662391,4.051372,1.137369,0.767839,0.483913,0.792176,1.727416,39.0
Geriatric Knee's and Buckets,16.464655,6.470292,4.689795,0.923919,0.756204,0.496149,0.768385,1.504770,39.0
TytheGreat,18.621570,5.750902,4.422630,1.158660,0.524387,0.478045,0.781999,1.802105,40.0
Dev,17.679109,5.711603,3.994457,1.024861,0.626832,0.464387,0.810582,1.947325,45.0


In [337]:
psa = pd.read_csv('/kaggle/input/preseason202526stats/preseason.csv', header=1)
print(psa.columns)

Index(['Rk', 'Player', 'Type', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'FG%', '3P%', 'FT%',
       '-9999'],
      dtype='object')


In [338]:
ps = psa[['Player', '3P', 'TRB', 'AST', 'STL', 'BLK', 'PTS', 'FG%', 'FT%']]
ps

,Player,3P,TRB,AST,STL,BLK,PTS,FG%,FT%
0,Precious Achiuwa,0.6,10.0,2.0,1.2,1.3,12.8,0.501,0.637
1,Steven Adams,0.3,13.6,3.1,1.0,1.2,11.3,0.532,0.479
2,Bam Adebayo,0.8,10.1,4.3,1.3,0.8,19.5,0.500,0.767
3,Ochai Agbaji,1.9,5.0,2.1,1.1,0.7,13.1,0.475,0.722
4,Santi Aldama,2.5,8.5,3.6,1.1,0.8,16.7,0.475,0.696
...,...,...,...,...,...,...,...,...,...
562,Delon Wright,1.3,4.4,4.8,2.1,0.6,9.6,0.395,0.740
563,Guerschon Yabusele,2.0,7.4,2.9,1.1,0.5,14.7,0.495,0.727
564,Jahmir Young,1.9,6.2,4.5,1.1,0.6,16.6,0.498,0.810
565,Trae Young,2.9,3.2,11.0,1.2,0.2,24.7,0.421,0.871


In [339]:
fbballa = fbball[['Team', 'Player', 'Position']]
fbballa

,Team,Player,Position
0,Geriatric Knee's and Buckets,Giannis Antetokounmpo,PF
1,Geriatric Knee's and Buckets,Trae Young,PG
2,Geriatric Knee's and Buckets,James Harden,PG
3,Geriatric Knee's and Buckets,Desmond Bane,SG
4,Geriatric Knee's and Buckets,Evan Mobley,PF
...,...,...,...
102,Dev,OG Anunoby,SF
103,Dev,Buddy Hield,SG
104,Dev,Jonas Valanciunas,C
105,Dev,Scoot Henderson,PG


In [340]:
merged_df = pd.merge(
    fbballa,
    ps,
    how='left',
    on='Player'
)
merged_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Team,Player,Position,3P,TRB,AST,STL,BLK,PTS,FG%,FT%
0,Geriatric Knee's and Buckets,Giannis Antetokounmpo,PF,0.4,12.0,6.6,1.0,1.1,31.2,0.593,0.632
1,Geriatric Knee's and Buckets,Trae Young,PG,2.9,3.2,11.0,1.2,0.2,24.7,0.421,0.871
2,Geriatric Knee's and Buckets,James Harden,PG,2.8,5.7,8.7,1.4,0.7,20.9,0.411,0.869
3,Geriatric Knee's and Buckets,Desmond Bane,SG,2.9,6.3,5.8,1.2,0.5,22.4,0.480,0.882
4,Geriatric Knee's and Buckets,Evan Mobley,PF,1.1,10.7,3.7,1.0,1.8,20.6,0.566,0.726
...,...,...,...,...,...,...,...,...,...,...,...
102,Dev,OG Anunoby,SF,2.2,4.8,2.3,1.5,0.8,17.2,0.478,0.803
103,Dev,Buddy Hield,SG,3.7,5.0,3.1,1.3,0.5,17.4,0.425,0.831
104,Dev,Jonas Valanciunas,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Dev,Scoot Henderson,PG,2.1,4.3,6.9,1.3,0.3,17.8,0.423,0.794


In [341]:
rows_with_missing_data = merged_df[merged_df.isnull().any(axis=1)]

# Display the result
print("Rows with Missing Values (Likely Name Mismatches):")
print(rows_with_missing_data)

Rows with Missing Values (Likely Name Mismatches):
                   Team              Player Position  3P  TRB  AST  STL  BLK  \
26           Da Man Dem        Nikola Jokic        C NaN  NaN  NaN  NaN  NaN   
71           Belt 2 Ass    Jimmy Butler III       SF NaN  NaN  NaN  NaN  NaN   
85   Keenerson’s Island      Alperen Sengun        C NaN  NaN  NaN  NaN  NaN   
90   Keenerson’s Island  Kristaps Porzingis        C NaN  NaN  NaN  NaN  NaN   
95                 Dev          Luka Doncic       PG NaN  NaN  NaN  NaN  NaN   
100                Dev       Nikola Vucevic        C NaN  NaN  NaN  NaN  NaN   
104                Dev    Jonas Valanciunas        C NaN  NaN  NaN  NaN  NaN   

     PTS  FG%  FT%  
26   NaN  NaN  NaN  
71   NaN  NaN  NaN  
85   NaN  NaN  NaN  
90   NaN  NaN  NaN  
95   NaN  NaN  NaN  
100  NaN  NaN  NaN  
104  NaN  NaN  NaN  


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [342]:
!pip install rapidfuzz
from rapidfuzz import process, fuzz

In [343]:
missing_names = [
    'Nikola Jokic',
    'Jimmy Butler III',
    'Alperen Sengun',
    'Kristaps Porzingis',
    'Luka Doncic',
    'Nikola Vucevic',
    'Jonas Valanciunas'
]

# 2. Get the list of all player names from 'ps' to search against
ps_names_list = ps['Player'].unique().tolist()

print("--- Searching 'ps' DataFrame for Closest Name Matches ---")
print("Use the 'Match in ps' names to create your correction map in the next step.")
print("-" * 50)

# Find the top match for each missing name
for missing_name in missing_names:
    # Use process.extractOne to get the single best match
    match, score, _ = process.extractOne(
        missing_name,
        ps_names_list,
        scorer=fuzz.ratio
    )
    print(f"Missing (fbball): '{missing_name}' -> Match (ps): '{match}' (Score: {score:.2f})")

--- Searching 'ps' DataFrame for Closest Name Matches ---
Use the 'Match in ps' names to create your correction map in the next step.
--------------------------------------------------
Missing (fbball): 'Nikola Jokic' -> Match (ps): 'Nikola Jokić' (Score: 91.67)
Missing (fbball): 'Jimmy Butler III' -> Match (ps): 'Jimmy Butler' (Score: 85.71)
Missing (fbball): 'Alperen Sengun' -> Match (ps): 'Alperen Şengün' (Score: 85.71)
Missing (fbball): 'Kristaps Porzingis' -> Match (ps): 'Kristaps Porziņģis' (Score: 88.89)
Missing (fbball): 'Luka Doncic' -> Match (ps): 'Luka Dončić' (Score: 81.82)
Missing (fbball): 'Nikola Vucevic' -> Match (ps): 'Nikola Vučević' (Score: 85.71)
Missing (fbball): 'Jonas Valanciunas' -> Match (ps): 'Jonas Valančiūnas' (Score: 88.24)


In [344]:
correction_map = {
    'Nikola Jokic': 'Nikola Jokić',         # Example correction from search output
    'Jimmy Butler III': 'Jimmy Butler',     # Example correction from search output
    'Alperen Sengun': 'Alperen Şengün',     # Example correction from search output
    'Kristaps Porzingis': 'Kristaps Porziņģis',    # Example correction from search output
    'Luka Doncic': 'Luka Dončić',           # Example correction from search output
    'Nikola Vucevic': 'Nikola Vučević',     # Example correction from search output
    'Jonas Valanciunas': 'Jonas Valančiūnas',         # Example correction from search output
}

# 1. Create a clean copy of your fbball DataFrame
fbball_cleaned = fbballa.copy()

# 2. Apply the name corrections
fbball_cleaned['Player'] = fbball_cleaned['Player'].replace(correction_map)

print("\n--- Name Correction Applied ---")

# Perform the final left merge
final_merged_df = pd.merge(
    fbball_cleaned,
    ps,
    how='left',
    on='Player',
)

# 5. Verify the fix by checking the corrected rows
corrected_players_final = list(correction_map.values())
verification_rows = final_merged_df[final_merged_df['Player'].isin(corrected_players_final)]

print("\n--- Final Merged Verification (NaNs should be filled) ---")
print(verification_rows.to_markdown(index=False))

# Optional: Check the total number of missing values now
print(f"\nTotal Missing Values remaining in merged columns: {final_merged_df[ps_stat_cols[1:]].isnull().sum().sum()}")


--- Name Correction Applied ---

--- Final Merged Verification (NaNs should be filled) ---
| Team               | Player             | Position   |   3P |   TRB |   AST |   STL |   BLK |   PTS |   FG% |   FT% |
|:-------------------|:-------------------|:-----------|-----:|------:|------:|------:|------:|------:|------:|------:|
| Da Man Dem         | Nikola Jokić       | C          |  1.6 |  12.3 |   9.6 |   1.6 |   0.7 |  27.7 | 0.578 | 0.806 |
| Belt 2 Ass         | Jimmy Butler       | SF         |  0.8 |   5.9 |   5.6 |   1.5 |   0.4 |  20.6 | 0.497 | 0.843 |
| Keenerson’s Island | Alperen Şengün     | C          |  0.5 |  11.4 |   5.5 |   1.3 |   0.9 |  22.2 | 0.52  | 0.704 |
| Keenerson’s Island | Kristaps Porziņģis | C          |  2.6 |   8.5 |   2.7 |   0.9 |   1.9 |  24   | 0.493 | 0.831 |
| Dev                | Luka Dončić        | PG         |  3.6 |   8.5 |   8.3 |   1.6 |   0.5 |  30.1 | 0.474 | 0.781 |
| Dev                | Nikola Vučević     | C          |  1.7 |  11.

In [345]:
final_merged_df

,Team,Player,Position,3P,TRB,AST,STL,BLK,PTS,FG%,FT%
0,Geriatric Knee's and Buckets,Giannis Antetokounmpo,PF,0.4,12.0,6.6,1.0,1.1,31.2,0.593,0.632
1,Geriatric Knee's and Buckets,Trae Young,PG,2.9,3.2,11.0,1.2,0.2,24.7,0.421,0.871
2,Geriatric Knee's and Buckets,James Harden,PG,2.8,5.7,8.7,1.4,0.7,20.9,0.411,0.869
3,Geriatric Knee's and Buckets,Desmond Bane,SG,2.9,6.3,5.8,1.2,0.5,22.4,0.480,0.882
4,Geriatric Knee's and Buckets,Evan Mobley,PF,1.1,10.7,3.7,1.0,1.8,20.6,0.566,0.726
...,...,...,...,...,...,...,...,...,...,...,...
102,Dev,OG Anunoby,SF,2.2,4.8,2.3,1.5,0.8,17.2,0.478,0.803
103,Dev,Buddy Hield,SG,3.7,5.0,3.1,1.3,0.5,17.4,0.425,0.831
104,Dev,Jonas Valančiūnas,C,0.5,13.8,3.5,0.7,1.1,19.2,0.543,0.835
105,Dev,Scoot Henderson,PG,2.1,4.3,6.9,1.3,0.3,17.8,0.423,0.794


In [346]:
final = final_merged_df.copy()
final['FT%'] = final['FT%'] * 100
final['FG%'] = final['FG%'] * 100
final

,Team,Player,Position,3P,TRB,AST,STL,BLK,PTS,FG%,FT%
0,Geriatric Knee's and Buckets,Giannis Antetokounmpo,PF,0.4,12.0,6.6,1.0,1.1,31.2,59.3,63.2
1,Geriatric Knee's and Buckets,Trae Young,PG,2.9,3.2,11.0,1.2,0.2,24.7,42.1,87.1
2,Geriatric Knee's and Buckets,James Harden,PG,2.8,5.7,8.7,1.4,0.7,20.9,41.1,86.9
3,Geriatric Knee's and Buckets,Desmond Bane,SG,2.9,6.3,5.8,1.2,0.5,22.4,48.0,88.2
4,Geriatric Knee's and Buckets,Evan Mobley,PF,1.1,10.7,3.7,1.0,1.8,20.6,56.6,72.6
...,...,...,...,...,...,...,...,...,...,...,...
102,Dev,OG Anunoby,SF,2.2,4.8,2.3,1.5,0.8,17.2,47.8,80.3
103,Dev,Buddy Hield,SG,3.7,5.0,3.1,1.3,0.5,17.4,42.5,83.1
104,Dev,Jonas Valančiūnas,C,0.5,13.8,3.5,0.7,1.1,19.2,54.3,83.5
105,Dev,Scoot Henderson,PG,2.1,4.3,6.9,1.3,0.3,17.8,42.3,79.4


In [347]:
team_agg_proj = final.groupby('Team').mean(numeric_only=True).round(1)
team_agg_proj

,3P,TRB,AST,STL,BLK,PTS,FG%,FT%
Team,,,,,,,,
Belt 2 Ass,1.9,6.9,4.6,1.3,0.9,20.1,48.9,79.0
Bryan's Brilliant Team,2.2,7.8,3.8,1.1,0.9,20.9,48.2,79.0
Da Man Dem,2.2,6.8,4.9,1.2,0.8,20.3,48.9,81.9
Dev,2.2,7.0,4.6,1.1,0.8,20.7,47.1,81.0
Geriatric Knee's and Buckets,1.8,8.1,5.3,1.1,1.0,19.6,49.8,77.6
Keenerson’s Island,2.1,6.8,4.8,1.2,0.7,19.9,47.9,78.3
Real Njggas do Real Things,2.4,7.4,4.3,1.0,0.7,21.6,49.5,80.1
TytheGreat,2.1,6.9,4.8,1.3,0.6,20.8,48.7,78.8


In [348]:
cats = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3P']

# Create rank columns for each category
ranked1 = team_agg_proj.copy()
for cat in cats:
    # ascending=False because higher value is better
    ranked1[cat + '_rank'] = ranked1[cat].rank(ascending=False, method='min')

# Calculate total rank score (sum of ranks)
rank_cols = [c + '_rank' for c in cats if c + '_rank' in ranked1.columns]
ranked1['TotalScore'] = ranked1[rank_cols].sum(axis=1)

# Optionally invert so higher = better (makes it easier to read)
ranked1['PowerScore'] = len(ranked1) * len(cats) - ranked1['TotalScore'] + 1

# Sort for final power rankings
ranked1 = ranked1.sort_values('PowerScore', ascending=False)

print("=== TEAM POWER RANKINGS BASED ON 2025-26 PROJECTIONS ===")
print(ranked1[['PowerScore', 'TotalScore']])

=== TEAM POWER RANKINGS BASED ON 2025-26 PROJECTIONS ===
                              PowerScore  TotalScore
Team                                                
Da Man Dem                          38.0        27.0
Bryan's Brilliant Team              34.0        31.0
Real Njggas do Real Things          34.0        31.0
Belt 2 Ass                          32.0        33.0
Geriatric Knee's and Buckets        32.0        33.0
Dev                                 31.0        34.0
TytheGreat                          29.0        36.0
Keenerson’s Island                  20.0        45.0


In [349]:
r1 = ranked1[['TotalScore', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3P']].copy()
r1

,TotalScore,PTS,TRB,AST,STL,BLK,FG%,FT%,3P
Team,,,,,,,,,
Da Man Dem,27.0,20.3,6.8,4.9,1.2,0.8,48.9,81.9,2.2
Bryan's Brilliant Team,31.0,20.9,7.8,3.8,1.1,0.9,48.2,79.0,2.2
Real Njggas do Real Things,31.0,21.6,7.4,4.3,1.0,0.7,49.5,80.1,2.4
Belt 2 Ass,33.0,20.1,6.9,4.6,1.3,0.9,48.9,79.0,1.9
Geriatric Knee's and Buckets,33.0,19.6,8.1,5.3,1.1,1.0,49.8,77.6,1.8
Dev,34.0,20.7,7.0,4.6,1.1,0.8,47.1,81.0,2.2
TytheGreat,36.0,20.8,6.9,4.8,1.3,0.6,48.7,78.8,2.1
Keenerson’s Island,45.0,19.9,6.8,4.8,1.2,0.7,47.9,78.3,2.1
